# RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval

This notebook shows how to use an implementation of RAPTOR with llama-index, leveraging the RAPTOR llama-pack.

RAPTOR works by recursively clustering and summarizing clusters in layers for retrieval.

There two retrieval modes:
- tree_traversal -- traversing the tree of clusters, performing top-k at each level in the tree.
- collapsed -- treat the entire tree as a giant pile of nodes, perform simple top-k.

See [the paper](https://arxiv.org/abs/2401.18059) for full algorithm details.

## Setup

In [30]:
!pip install llama-index llama-index-packs-raptor llama-index-vector-stores-qdrant chromadb  llama-index-vector-stores-chroma

Defaulting to user installation because normal site-packages is not writeable


In [31]:
from llama_index.packs.raptor import RaptorPack

# optionally download the pack to inspect/modify it yourself!
# from llama_index.core.llama_pack import download_llama_pack
# RaptorPack = download_llama_pack("RaptorPack", "./raptor_pack")

In [22]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-cHgnNwpihYLM884UqVkDT3BlbkFJGw0AgW8etJSuz6s13Le1"

## Constructing the Clusters/Hierarchy Tree

In [23]:
import nest_asyncio

nest_asyncio.apply()

In [24]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["./Form Master Services Agreement (Outsourcing).DOCX"]).load_data()

In [32]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore # type: ignore
import chromadb

client = chromadb.PersistentClient(path="./raptor_paper_db")
collection = client.get_or_create_collection("raptor")

vector_store = ChromaVectorStore(chroma_collection=collection)

raptor_pack = RaptorPack(
    documents,
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ),  # used for embedding clusters
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),  # used for generating summaries
    vector_store=vector_store,  # used for storage
    similarity_top_k=2,  # top k for each layer, or overall top-k for collapsed
    mode="collapsed",  # sets default mode
    transformations=[
        SentenceSplitter(chunk_size=400, chunk_overlap=50)
    ],  # transformations applied for ingestion
)

Generating embeddings for level 0.
Performing clustering for level 0.


/home/harish/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


Generating summaries for level 0 with 24 clusters.
Level 0 created summaries/clusters: 24
Generating embeddings for level 1.
Performing clustering for level 1.
Generating summaries for level 1 with 4 clusters.
Level 1 created summaries/clusters: 4
Generating embeddings for level 2.
Performing clustering for level 2.
Generating summaries for level 2 with 1 clusters.
Level 2 created summaries/clusters: 1


## Retrieval

In [34]:
nodes = raptor_pack.run("What are the intellectual property rights of the vendor?", mode="collapsed")
print(len(nodes))
print(nodes[0].text)

2
The agreement outlines the terms related to changes in charges, intellectual property rights, ownership of work products, disclosure of inventions, and software provisions between the Provider and the Client. It specifies that any changes in charges must be agreed upon and adjusted accordingly. The agreement also addresses the ownership of software, work products, and modifications, with the Client retaining all rights and interests. It further discusses the use of Provider Proprietary Materials and the disclosure of inventions made during the agreement. Additionally, it covers the installation of upgrades and modifications to software, as well as the granting of rights and licenses related to Provider Patents within specific industries.


In [35]:
nodes = raptor_pack.run(
    "What are the intellectual property rights of the vendor?", mode="tree_traversal"
)
print(len(nodes))
print(nodes[0].text)

Retrieved parent IDs from level 2: ['0f0c386b-12f5-460c-9fb2-5d65b0024c7c']
Retrieved 2 from parents at level 2.
Retrieved parent IDs from level 1: ['750e2166-637d-4a6a-8b6c-86b69ba6ad97', 'd6ead990-a89c-420d-b1bb-8108516f4b8f']
Retrieved 4 from parents at level 1.
Retrieved parent IDs from level 0: ['a9cc1f53-70ae-4e05-bfb5-e409ccd914f6', '637bef95-1443-4125-8ae0-4a7793896c0a']
Retrieved 4 from parents at level 0.
4
The territorial extent of the rights in the Work Product assigned to [XXXXXXXX] by Provider and/or the Provider Personnel under this Agreement shall extend to all the countries in the world.  The assignment of the Intellectual Property Rights in the Work Product by Provider and/or the Provider Personnel to [XXXXXXXX] shall be royalty-free absolute, irrevocable and perpetual.  

With respect to any Services performed in India, the Parties agree that, without limitation of any other [XXXXXXXX] rights or remedies under the Agreement, the following provisions shall apply: (i) 

## Loading

Since we saved to a vector store, we can also use it again! (For local vector stores, there is a `persist` and `from_persist_dir` method on the retriever)

In [37]:
from llama_index.packs.raptor import RaptorRetriever

retriever = RaptorRetriever(
    [],
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ),  # used for embedding clusters
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),  # used for generating summaries
    vector_store=vector_store,  # used for storage
    similarity_top_k=2,  # top k for each layer, or overall top-k for collapsed
    mode="tree_traversal",  # sets default mode
)

In [ ]:
# if using a default vector store
# retriever.persist("./persist")
# retriever = RaptorRetriever.from_persist_dir("./persist", ...)

## Query Engine

In [38]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever, llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1)
)

In [41]:
response = query_engine.query("Explain how termination works in the agreement?")

In [42]:
print(str(response))

Termination in the agreement can occur for various reasons such as expiration, termination for convenience, termination for cause, termination due to breach, termination for damages cap exceeded, or termination for non-payment. In the event of termination, certain obligations and liabilities of both parties may continue, and termination may lead to the automatic termination of all work orders then in effect. Additionally, there are provisions for termination assistance to facilitate the orderly transfer of services back to the provider or its designee, with charges for such assistance specified in the agreement.
